# Import Libraries

In [1]:
! nvidia-smi

Sat Jun 17 02:44:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

###  Target:
    - LR scheduler
    - Adam optimizer
    - Fine tune number of channels in nodes

### Results:
    - Parameters: 7.9 K
    - Best Train Accuracy: 99.01
    - Best Test Accuracy: 99.25

### Analsis:
    - Model is learning faster now. 
    - More channels in the inital layers seems to perform better
    - Data augumentation might improve the performance.

## Data Transformations


In [3]:
# Train Phase transformations
train_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

# Test Phase transformations
test_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])


# Dataset and Creating Train/Test Split

In [4]:
from utils import prepare_mnist_data
train_loader, test_loader = prepare_mnist_data(
    train_transforms, test_transforms, batch_size=256)

CUDA Available? False


# Dataloader Arguments & Test/Train Dataloaders


In [5]:
!pip install torchsummary
from torchsummary import summary
from model import Net
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net(
    list('CcCcPCCCCGc'),
    [32, 16, 16, 10, 10,
     16, 8, 8, 16,
     16, 10], 
    dropout_value=0.03).to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
              ReLU-2           [-1, 32, 28, 28]               0
       BatchNorm2d-3           [-1, 32, 28, 28]              64
           Dropout-4           [-1, 32, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]             512
            Conv2d-6           [-1, 16, 28, 28]           2,304
              ReLU-7           [-1, 16, 28, 28]               0
       BatchNorm2d-8           [-1, 16, 28, 28]              32
           Dropout-9           [-1, 16, 28, 28]               0
           Conv2d-10           [-1, 10, 28, 28]             160
        MaxPool2d-11           [-1, 10, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           1,440
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1,

# Training and Testing

In [19]:
from torch.optim.lr_scheduler import StepLR
from model import train, test
model =  Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=4, gamma=0.05)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loss=0.1647590845823288 Batch_id=468 Accuracy=93.11: 100%|██████████| 469/469 [00:18<00:00, 25.17it/s]



Test set: Average loss: 0.0832, Accuracy: 9751/10000 (97.51%)

EPOCH: 1


Loss=0.06970038264989853 Batch_id=468 Accuracy=97.53: 100%|██████████| 469/469 [00:17<00:00, 27.00it/s]



Test set: Average loss: 0.0407, Accuracy: 9869/10000 (98.69%)

EPOCH: 2


...


Test set: Average loss: 0.0268, Accuracy: 9915/10000 (99.15%)

EPOCH: 18


Loss=0.012235316447913647 Batch_id=468 Accuracy=99.01: 100%|██████████| 469/469 [00:17<00:00, 27.16it/s]



Test set: Average loss: 0.0253, Accuracy: 9916/10000 (99.16%)

EPOCH: 19


Loss=0.006576744373887777 Batch_id=468 Accuracy=98.95: 100%|██████████| 469/469 [00:17<00:00, 26.85it/s]



Test set: Average loss: 0.0280, Accuracy: 9918/10000 (99.18%)

